<a href="https://colab.research.google.com/github/vivekjadhav0606/Stockmarket-Prediction/blob/main/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gradio as gr
import pandas as pd
import yfinance as yf
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import datetime

# 1. CSV Load karein
try:
    df_meta = pd.read_csv('/content/drive/MyDrive/stockmarketing/symbols_valid_meta.csv')
    print("CSV file loaded successfully!")
except Exception as e:
    print(f"Error: CSV file nahi mili. Please upload 'symbols_valid_meta.csv'. {e}")

def get_prediction(symbol):
    try:
        symbol = symbol.upper().strip()

        # 2. Search Symbol in CSV
        match = df_meta[df_meta['Symbol'] == symbol]
        if match.empty:
            return "Symbol not found in CSV! Please try another.", None

        stock_name = match.iloc[0]['Security Name']

        # 3. Download Data (Fixed MultiIndex handling)
        data = yf.download(symbol, period="2y", interval="1d")
        if data.empty or len(data) < 60:
            return "Data not available or not enough history (min 60 days).", None

        # Fix: Yfinance multi-index columns ko handle karne ke liye
        # Agar MultiIndex hai toh sirf Close column select karein
        df_close = data['Close'].values.reshape(-1, 1)

        # 4. Scaling
        scaler = MinMaxScaler(feature_range=(0,1))
        scaled_data = scaler.fit_transform(df_close)

        # Training logic (Simplified for fast dashboard response)
        x_train, y_train = [], []
        for i in range(60, len(scaled_data)):
            x_train.append(scaled_data[i-60:i, 0])
            y_train.append(scaled_data[i, 0])

        x_train, y_train = np.array(x_train), np.array(y_train)
        x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

        # 5. Build and Train Model (Small epochs for Gradio speed)
        model = Sequential([
            LSTM(50, return_sequences=True, input_shape=(60, 1)),
            Dropout(0.2),
            LSTM(50, return_sequences=False),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, epochs=2, batch_size=32, verbose=0)

        # 6. Predict Tomorrow's Price
        last_60_days = scaled_data[-60:]
        X_test = np.reshape(np.array([last_60_days]), (1, 60, 1))
        pred_scaled = model.predict(X_test)
        predicted_price = scaler.inverse_transform(pred_scaled)[0][0]

        # Current Stats
        current_price = df_close[-1][0]

        # 7. Create Plotly Graph
        fig = go.Figure()
        # Historical Line
        fig.add_trace(go.Scatter(
            x=data.index[-100:],
            y=df_close[-100:].flatten(),
            name="Historical Price",
            line=dict(color='#00ffcc', width=2)
        ))

        # Predicted Point
        tomorrow_date = data.index[-1] + datetime.timedelta(days=1)
        fig.add_trace(go.Scatter(
            x=[tomorrow_date],
            y=[predicted_price],
            mode='markers+text',
            name="Tomorrow's Prediction",
            marker=dict(color='orange', size=15, symbol='star'),
            text=[f"Predicted: ${predicted_price:.2f}"],
            textposition="top center"
        ))

        fig.update_layout(
            title=f"{stock_name} ({symbol}) Price Forecast",
            template="plotly_dark",
            xaxis_title="Date",
            yaxis_title="Price (USD)",
            hovermode="x unified"
        )

        summary = (f"✅ Stock: {stock_name}\n"
                   f"📈 Current Price: ${current_price:.2f}\n"
                   f"🚀 Tomorrow's Predicted Price: ${predicted_price:.2f}\n"
                   f"📅 Date: {tomorrow_date.strftime('%Y-%m-%d')}")

        return summary, fig

    except Exception as e:
        return f"Error occurred: {str(e)}", None

# 8. Gradio Interface Construction
with gr.Blocks(theme=gr.themes.Default(primary_hue="emerald")) as demo:
    gr.Markdown("# 📈 Stock Market Real-Time Predictor")
    gr.Markdown("Search stocks from your CSV and predict tomorrow's closing price using LSTM.")

    with gr.Row():
        ticker_input = gr.Textbox(label="Enter Symbol (e.g., AAPL, TSLA, MSFT)", placeholder="Search...")
        submit_btn = gr.Button("Search & Predict", variant="primary")

    with gr.Column():
        result_info = gr.Textbox(label="Market Summary", lines=5)
        result_plot = gr.Plot(label="Price Movement Visualization")

    submit_btn.click(fn=get_prediction, inputs=ticker_input, outputs=[result_info, result_plot])

# Launch the app
demo.launch(debug=True)

CSV file loaded successfully!


/tmp/ipython-input-81266200.py:111: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Default(primary_hue="emerald")) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b331da294a2910e5fb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/tmp/ipython-input-81266200.py:30: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, period="2y", interval="1d")
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step


/tmp/ipython-input-81266200.py:30: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AAN']: YFPricesMissingError('possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")')
/tmp/ipython-input-81266200.py:30: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
